In [1]:
import torch
import torch.nn.functional as F
from torch import fx
from torch.nn import Module
import time
import tvm
from tvm import relax
import tvm.testing
from tvm.script import ir as I
from tvm.script import relax as R
from tvm.script import tir as T
from tvm import relay
from tvm.relay.testing import *
from tvm.relay.testing import layers
from tvm.relay.testing.resnet import *
from tvm.contrib import graph_executor
from tvm import relax
from tvm.relax import testing
from tvm.relax.testing import relay_translator, nn

In [2]:

units = [48//6] * 3
filter_list = [16, 16, 32, 64]
num_stages=3
bottle_neck = False
shape =( 1,3,224,224)
layout="NCHW"
dtype="float32"
kernel_layout = "OIHW" if layout == "NCHW" else "HWIO"
bn_axis = layout.index("C")

In [3]:

def get3D():
    body = relay.var("data", shape=(1,1,3,224,224), dtype=dtype)
    for i in range(25):
        body = layers.conv3d(
                data=body,
                channels=filter_list[0],
                kernel_size=(3, 7, 7),
                strides=(1, 2, 2),
                padding=(1, 3, 3),
                name="conv0",
                data_layout="NCDHW",
                kernel_layout="OIDHW",
            )
        body = layers.batch_norm_infer(data=body, epsilon=2e-5, name="bn0")
        body = relay.nn.relu(data=body)
    f = relay.Function(relay.analysis.free_vars(body), body)
    return f

In [3]:
def getCBR():
    name = "a1"
    data = relay.var("data", shape=(1,3,224,224), dtype=dtype)
    body = layers.conv2d(
            data=data,
            channels=filter_list[0],
            kernel_size=(7, 7),
            strides=(2, 2),
            padding=(3, 3),
            name="conv0",
            data_layout=layout,
            kernel_layout=kernel_layout,
        )
    body = layers.batch_norm_infer(data=body, epsilon=2e-5, axis=bn_axis, name="bn0")
    body = relay.nn.relu(data=body)
    body = relay.nn.max_pool2d(
                data=body, pool_size=(3, 3), strides=(2, 2), padding=(1, 1), layout=layout
            )
    for i in range(100):
        body = residual_unit(
                    body,
                    filter_list[0],
                    (1, 1),
                    True,
                    name=f"stage{i + 1}_unit{i + 2}",
                    bottle_neck=bottle_neck,
                    data_layout=layout,
                    kernel_layout=kernel_layout,
                )

    f = relay.Function(relay.analysis.free_vars(body), body)
    return f

In [20]:
def getCAR():
    name = "a2"
    body = relay.var("data", shape=(1,3,224,224), dtype=dtype)
   
    # body = layers.batch_norm_infer(data=body, epsilon=2e-5, axis=bn_axis, name="bn0")
    body = relay.nn.relu(data=body)
    # body = relay.nn.max_pool2d(
    #             data=body, pool_size=(3, 3), strides=(2, 2), padding=(1, 1), layout=layout
    #         )
    for i in range(50):
        body = layers.conv2d(
            data=body,
            channels=filter_list[0],
            kernel_size=(7, 7),
            strides=(2, 2),
            padding=(3, 3),
            name="conv0",
            data_layout=layout,
            kernel_layout=kernel_layout,
        )
        body = relay.nn.bias_add(body,relay.var("conv1_bias"))
        body = relay.nn.relu(body)

    f = relay.Function(relay.analysis.free_vars(body), body)
    return f

In [3]:
def getCR():
    name = "a2"
    body = relay.var("data", shape=(1,3,224,224), dtype=dtype)
   
    # body = layers.batch_norm_infer(data=body, epsilon=2e-5, axis=bn_axis, name="bn0")
    body = relay.nn.relu(data=body)
    # body = relay.nn.max_pool2d(
    #             data=body, pool_size=(3, 3), strides=(2, 2), padding=(1, 1), layout=layout
    #         )
    for i in range(50):
        body = layers.conv2d(
            data=body,
            channels=filter_list[0],
            kernel_size=(7, 7),
            strides=(2, 2),
            padding=(3, 3),
            name="conv0",
            data_layout=layout,
            kernel_layout=kernel_layout,
        )
        body = relay.nn.relu(body)

    f = relay.Function(relay.analysis.free_vars(body), body)
    return f

In [3]:
def getDAB():
    data = relay.var("data", relay.TensorType((1,  224), dtype))
   
    body = layers.dense_add_bias(data=data, units=4096, name="fc6")
    for i in range(25):
        body = layers.dense_add_bias(body, units=4096, name="fc6")

    f = relay.Function(relay.analysis.free_vars(body), body)
    return f

In [1]:
def getDABR():
    data = relay.var("data", relay.TensorType((1,  224), dtype))
   
    body = layers.dense_add_bias(data=data, units=4096, name="fc6")
    for i in range(50):
        body = layers.dense_add_bias(body, units=4096, name="fc6")
        body = relay.nn.relu(body)

    f = relay.Function(relay.analysis.free_vars(body), body)
    return f

In [4]:
mod = get3D()

In [5]:
shape = (1,3, 224, 224)

In [148]:
# with tvm.transform.PassContext(opt_level=0):
#     lib = relay.build(mod, "llvm")



# dev = tvm.cpu(0)
# dtype = "float32"
# m = graph_executor.GraphModule(lib["default"](dev))
# m.set_input("data", tvm.nd.array(np.random.randn(1,112)))
# m.module.time_evaluator("run", tvm.cpu())().mean * 1000

501.6899684

In [15]:
with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(mod, "cuda")

dev = tvm.cuda(0)
dtype = "float32"
m = graph_executor.GraphModule(lib["default"](dev))



In [12]:
m.set_input("data", tvm.nd.array(np.random.randn(1,1, 3,224,112)))
m.module.time_evaluator("run", tvm.cuda())().mean*1000

99.64254759999999

In [16]:
m.module.time_evaluator("run", tvm.cpu())().mean*1000

70.81684699999998

In [6]:
relax_mod = relay_translator.from_relay(mod, "cuda")

In [10]:
with tvm.target.Target("cuda"):
    relax_mod = relax.transform.LegalizeOps()(relax_mod)
    relax_mod = tvm.tir.transform.DefaultGPUSchedule()(relax_mod)
        # seq = tvm.transform.Sequential(
        #     [relax.transform.LegalizeOps(), tvm.tir.transform.DefaultGPUSchedule()]
        # )
        # relax_mod = seq(relax_mod)

ex = relax.build(relax_mod, "cuda")
vm = relax.VirtualMachine(ex, tvm.cuda())

data = tvm.nd.array(np.random.randn(1,1,3,224,224).astype(np.float32))
params = nn.init_params(relax_mod)

vm.save_function("main","base_func", data, *params)
vm.time_evaluator("base_func", tvm.cuda())().mean * 1000

3.2055454

In [41]:
vm.time_evaluator("base_func", tvm.cpu())().mean * 1000

0.1105678

In [9]:

with tvm.transform.PassContext(opt_level=3):
  # relay_mod = relay.transform.SimplifyInference()(mod)
  # relay_mod = relay.transform.FoldConstant()(mod)
  # relay_mod = relay.transform.FoldScaleAxis()(relay_mod)
  # relay_mod = relay.transform.CanonicalizeOps()(relay_mod)
  # relay_mod = relay.transform.AlterOpLayout()(relay_mod)
  # relay_mod = relay.transform.FoldConstant()(relay_mod)
  
  relax_mod = relay_translator.from_relay(mod, "cuda")
#   relax_mod = relax.transform.AnnotateTIROpPattern()(relax_mod)
  relax_mod = relax.transform.FuseOps()(relax_mod)
#   relax_mod = relax.transform.FuseTIR()(relax_mod)
#   relax_mod = relax.transform.DecomposeOpsForInference()(relax_mod)
#   relax_mod = relax.transform.LegalizeOps()(relax_mod)


In [ ]:
from tvm import dlight as dl
from tvm.dlight.benchmark import (
    benchmark,
    benchmark_prim_func,
    benchmark_relax_func,
    extract_prim_func,
    extract_from_relax,
    extract_func_info_from_prim_func,
)

# with tvm.target.Target("cuda"):
#     benchmark_relax_func(relax_mod, "main")

with tvm.target.Target("cuda"):
    d_cuda_mod = dl.ApplyDefaultSchedule(
        # dl.gpu.Matmul(),
        # dl.gpu.Transpose(),
        dl.gpu.Reduction(),
        # dl.gpu.Transpose(),
        dl.gpu.DecodeGEMV(),
        # dl.gpu.Matmul(),
        # dl.gpu.Fallback(),
    )(relax_mod)

In [10]:
ex = relax.build(d_cuda_mod, "cuda")
vm = relax.VirtualMachine(ex, tvm.cuda())

shape = (1, 1,3,224,224)
data = tvm.nd.array(np.random.rand(*shape).astype(np.float32))
params = nn.init_params(d_cuda_mod)

vm.save_function("main","base_func", data, *params)
vm.time_evaluator("base_func", tvm.cuda())().mean * 1000

TVMError: Traceback (most recent call last):
  5: 0x000055e1dfa4e250
  4: operator()
        at /root/wang/tvm/src/driver/driver_api.cc:514
  3: tvm::TIRToRuntime(tvm::runtime::Map<tvm::Target, tvm::IRModule, void, void> const&, tvm::Target const&)
        at /root/wang/tvm/src/driver/driver_api.cc:475
  2: tvm::SplitMixedModule(tvm::IRModule, tvm::Target const&, tvm::Target const&)
        at /root/wang/tvm/src/driver/driver_api.cc:419
  1: tvm::ApplyPasses(tvm::IRModule, tvm::transform::Sequential)
        at /root/wang/tvm/src/driver/driver_api.cc:290
  0: operator()
        at /root/wang/tvm/src/tir/analysis/verify_memory.cc:205
  Did you forget to bind?
    Variable `T_relu` is directly accessed by host memory (it is not contained in a thread environment or in the function arguments.
    Variable `bn0_beta` is directly accessed by host memory (it is not contained in a thread environment or in the function arguments.
    Variable `lv2` is directly accessed by host memory (it is not

In [17]:
vm.time_evaluator("base_func", tvm.cpu())().mean * 1000


5.4727045